In [8]:
%load_ext autoreload
%autoreload 2
import os
from df2d.dataset import Drosophila2Dataset
from df2d.model import Drosophila2DPose
from df2d.parser import create_parser
from df2d.util import pwd
from df2d.inference import path2inp, pr2inp
import torch
from torch.utils.data import DataLoader
import numpy as np


import matplotlib.pyplot as plt
from df2d.inference import parse_img_path

class load_f:
    def __init__(self, cam_order):
        self.cam_order = cam_order.tolist()
    def __call__(self, x):
        img = plt.imread(x)
        cam_id, _ = parse_img_path(x)
        if self.cam_order.index(cam_id) > 3:
            img = img[:, ::-1]
        return img

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint_path = os.path.join(pwd(), "../weights/sh8_deepfly.tar")
args_default = create_parser().parse_args("").__dict__
model = Drosophila2DPose(checkpoint_path=checkpoint_path, **args_default).to(device)

inp = pr2inp('/home/user/Desktop/DeepFly3D/sample/test/df3d/df3d_result__home_user_Desktop_DeepFly3D_sample_test.pkl')

#inp = path2inp(
#    '/home/user/Desktop/DeepFly3D/sample/test/', max_img_id=100
#)  # extract list of images under the folder
dat = DataLoader(Drosophila2Dataset(inp, return_heatmap=True, load_f=load_f(np.array([0,1,2,3,4,5,6]))), batch_size=8, shuffle=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from pytorch_lightning import Trainer

trainer = Trainer()
trainer.fit(model, dat)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type         | Params
---------------------------------------
0 | model | HourglassNet | 6.7 M 
---------------------------------------
6.7 M     Trainable params
0         Non-trainable params
6.7 M     Total params
26.933    Total estimated model params size (MB)
Epoch 1:  75%|███████▌  | 9/12 [00:35<00:11,  3.90s/it, loss=0.00381, v_num=67]